In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import utils as u
modelname = "KB/bert-base-swedish-cased"

/home/mun/miniconda3/envs/rcon/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import utils as u
labeled, unlabeled = u.read_data("../data/TRVInfra_all_only_requirements_202312.xlsx")

Loaded 21316 sentences.
Labeled sentences:   3000
Unlabeled sentences: 18316
Label 0.0 with 479 instances is the smaller group.


In [4]:
sentences = u.get_sentences(labeled)
labels = u.get_labels(labeled) 

In [5]:
input_ids, attention_masks = u.tokenize(modelname, sentences)

Max sentence length: 156


/home/mun/miniconda3/envs/rcon/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [6]:
train_dataset, val_dataset, test_dataset = u.create_datasets(input_ids, attention_masks, labels, 0.1, 0.9)

  958 total samples
  775 training samples
   87 validation samples
   96 test samples


In [8]:
model = u.train(modelname, 3, train_dataset, val_dataset)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at KB/bert-base-swedish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/mun/miniconda3/envs/rcon/lib/python3.12/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


There are 2 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2080 Ti

======== Epoch 1 / 3 ========
Training...

  Average training loss: 0.29
  Training epoch took: 0:00:08

Running Validation...
  Accuracy: 0.97
  Validation Loss: 0.09
  Validation took: 0:00:00

======== Epoch 2 / 3 ========
Training...

  Average training loss: 0.10
  Training epoch took: 0:00:07

Running Validation...
  Accuracy: 0.99
  Validation Loss: 0.06
  Validation took: 0:00:00

======== Epoch 3 / 3 ========
Training...

  Average training loss: 0.05
  Training epoch took: 0:00:07

Running Validation...
  Accuracy: 0.99
  Validation Loss: 0.06
  Validation took: 0:00:00

Training complete!
Total training took 0:00:23 (h:mm:ss)

Training stats
       Training Loss  Valid. Loss  Valid. Accur. Training Time Validation Time
epoch                                                                         
1               0.29         0.09           0.97       0:00:08         0:00:00
2               0.10   

In [9]:
u.evaluate(model, test_dataset)

Predicting labels for 96 test sentences...
There are 2 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2080 Ti
    DONE.
Positive samples: 54 of 96 (56.25%)

Calculating Matthews Corr. Coef. for each batch...
Total MCC: 0.958
Accuracy: 0.979167


In [20]:
u.save_model(model, "../models")

# Prediction on unlabeled data

In [47]:
modelpath = "../models/20240312-202620"
model = u.load_model(modelpath)

There are 2 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2080 Ti


In [28]:
sentences = u.get_sentences(unlabeled)
ids = u.get_ids(unlabeled)
input_ids, attention_masks = u.tokenize(modelname, sentences)

Max sentence length: 321


/home/mun/miniconda3/envs/rcon/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [39]:
predictions = u.predict(model, u.create_dataset(input_ids, attention_masks))

There are 2 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2080 Ti
  Batch    50  of    573.    Elapsed: 0:00:11.
  Batch   100  of    573.    Elapsed: 0:00:22.
  Batch   150  of    573.    Elapsed: 0:00:34.
  Batch   200  of    573.    Elapsed: 0:00:46.
  Batch   250  of    573.    Elapsed: 0:00:58.
  Batch   300  of    573.    Elapsed: 0:01:10.
  Batch   350  of    573.    Elapsed: 0:01:23.
  Batch   400  of    573.    Elapsed: 0:01:35.
  Batch   450  of    573.    Elapsed: 0:01:48.
  Batch   500  of    573.    Elapsed: 0:02:01.
  Batch   550  of    573.    Elapsed: 0:02:14.
    DONE.


In [49]:
u.save_predictions(predictions, ids, sentences, modelpath)

Predictions have been stored in ../models/20240312-202620/predictions_20240312-214127.csv
